In [ ]:
import numpy as np
import math as mt
import random
import pandas as pd

In [ ]:
data = pd.read_excel('Load Data in kW.xlsx')
data.head()

In [ ]:
x = []
y = []
for i in range(24 * 7, data.shape[0]):
    x.append(data['Load (kW)'].iloc[i - 24 * 7])
    y.append(data['Load (kW)'].iloc[i])
data = pd.DataFrame({'Previous Week Load' : x, 'Present Week Load' : y})
data.rename(columns={'Previous Week Load': 'x',
                   'Present Week Load': 'y'},
          inplace=True)
data.head()

In [ ]:
x_min = data['x'].min()
x_max = data['x'].max()

y_min = data['y'].min()
y_max = data['y'].max()

In [ ]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
num_vars = ['x', 'y']
data[num_vars] = scaler.fit_transform(data[num_vars])
data.head()

In [ ]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(data.x, data.y, test_size = 0.20, random_state = 101)

In [ ]:
m = random.uniform(-10,10)
c = random.uniform(-10,10)
eta = 0.1
epochs = 1000
c_vel = 0.9
em = 0
ec = 0
epsilon = 1E-6

for iter in range(epochs):
  for sample in range(x_train.shape[0]):
    
    m_deriv = -1 * (y_train.iloc[sample] - m * x_train.iloc[sample] - c) * x_train.iloc[sample]
    c_deriv = -1 * (y_train.iloc[sample] - m * x_train.iloc[sample] - c)
    em = (c_vel * em) + ((1 - c_vel) * (m_deriv ** 2)) 
    ec = (c_vel * ec) + ((1 - c_vel) * (c_deriv ** 2)) 
    eta_m = (eta/(mt.sqrt(em + epsilon))) * m_deriv
    eta_c = (eta/(mt.sqrt(ec + epsilon))) * c_deriv
    m = m - eta_m
    c = c - eta_c

In [ ]:
y_pred_train = []
for i in x_train:
  y_p_tr = (m * i) + c
  y_pred_train.append(y_p_tr)
    
y_pred_train = np.array(y_pred_train)
y_pred_test = []
for i in x_test:
  y_p_te = (m * i) + c
  y_pred_test.append(y_p_te)
y_pred_test = np.array(y_pred_test)

In [ ]:
import math
from sklearn.metrics import mean_squared_error 
from sklearn.metrics import mean_absolute_error

print('TRAINING ERRORS\n')
rmse = math.sqrt(mean_squared_error(y_train, y_pred_train)) 
print('Root mean square error', rmse) 
mse = (mean_squared_error(y_train, y_pred_train)) 
print('Mean square error', mse) 
mae=mean_absolute_error(y_train, y_pred_train)
print('Mean absolute error', mae)

In [ ]:
mse = math.sqrt(mean_squared_error(y_test, y_pred_test)) 
print('TESTING ERRORS\n')
print('Root mean square error', mse) 
mse = (mean_squared_error(y_test, y_pred_test)) 
print('Mean square error', mse) 
mae=mean_absolute_error(y_test, y_pred_test)
print('Mean absolute error', mae)

In [ ]:
x = float(input('Enter input value'))
print('X=',x)

x_nor = (x - x_min)/(x_max - x_min)
y_nor = m*float(x_nor) + c

Y = y_nor*(y_max - y_min) + y_min
print('Output = ',Y)